In [ ]:
import pandas as pd
import polars as pl
import numpy as np
import pickle, warnings, datetime
warnings.filterwarnings('ignore')

In [120]:
# Determine which times to keep based on the stock data
with open(r"../../Data/Processed/stock_data_simple.pkl", 'rb') as f:
    stock_data = pickle.load(f)
times = list(stock_data.index.unique())

In [121]:
df = pl.read_csv('../../data/processed/gdelt_combined_20250609_1222.csv',
    columns=[
        'GKGRECORDID',
        'V2SOURCECOLLECTIONIDENTIFIER',
        'V2SOURCECOMMONNAME',
        'V2DOCUMENTIDENTIFIER',
        'V1THEMES',
        'V2.1TRANSLATIONINFO',
        'datetime',
        'date',
        'airplane',
        'airline',
        'airport',
        'Alaska Airlines',
        'American Airlines',
        'Delta Air Lines',
        'Frontier Airlines',
        'Hawaiian Airlines',
        'JetBlue',
        'Southwest Airlines',
        'Spirit Airlines',
        'Sun Country Airlines',
        'United Airlines',
        'Allegiant Air',
        'article_title',
        'Tone',
        'Positive Score',
        'Negative Score',
        'Polarity',
        'Activity Reference Density',
        'Self/Group Reference Density',
        'Word Count']
    )

In [122]:
df = df.select(pl.all().shrink_dtype())

In [123]:
# Remove a list of titles that are probably home pages or other non-article content
titles_to_remove = [
    'News briefs',
    'Latest Articles',
    'The Nashville Ledger',
    'Today in History',
    'National News - Media One Radio Group (WWSE | WJTN | WHUG | WKSN | WQFX',
    "Aero-News Network: The aviation and aerospace world's daily/real-time news and information service",
    'Drake & 21 Savage Add More Texas Concert Dates Due To High Demand',
    'Stock Market | FinancialContent Business Page',
    'Radio Station WHMI 93.5 FM &#x2014; Livingston County Michigan News, Weather, Traffic, Sports, School Updates, and the Best Classic Hit',
    'National - KSYL-AM',
    'Business Highlights',
    'Business Highlights',
    'National News - 1540 WADK Newport',
    'National - Carroll Broadcasting Inc.',
    'ABC National - WOND',
    "Breaking National News - 92.7-FM TheDRIVE - Bob & Tom Mornings, Central New York's Best Rock All Day",
    "ABC - National News - Xtra 99.1 FM - Today's Hits and Yesterday's Favorites",
    "Ed Bruce, Legendary Country Songwriter, 'Maverick' Actor, Dead At 81",
    "ABC National News - Beach 95.1 - WBPC Panama City Beach Greatest Hits of the 60s, 70s & 80s",
    "CES gadget show: How watching TV will change in the 2020s",
    "Despite business warnings, GOP moves ahead with voting bills",
    "KTBB.com - News Weather Talk",
    "AP Story",
    "SRN - US News - Taylorville Daily News",
]

# Replace empty article titles with z
df = df.with_columns(
    pl.col('article_title').fill_null('z')
)

df = df.filter(
    ~df['article_title'].is_in(titles_to_remove)
)

# Drop where article_title contains 'AP News in Brief'
df = df.filter(
    ~df['article_title'].str.contains('AP News in Brief at', literal=True)
    )

In [124]:
site_counts = df['V2SOURCECOMMONNAME'].value_counts().to_pandas()
site_counts

,V2SOURCECOMMONNAME,count
0,ajot.com,428
1,kkgb.com,31
2,thewolf.ca,31
3,warsawvoice.pl,1
4,queenslandcountrylife.com.au,1
...,...,...
13802,thewhig.com,13
13803,greenvillejournal.com,3
13804,1310news.com,352
13805,standard.co.uk,104


In [126]:
# Export the GKGRECORDID field to a CSV file
df[['GKGRECORDID']].to_csv('../../data/processed/gdelt_kept_records.csv', index=False)

In [ ]:
# Save df to pickle file
with open('../../data/processed/gdelt_intermediate_cleaned.pkl', 'wb') as f:
    pickle.dump(df, f)